## Loading the dataset

In [ ]:
#!pip install sentence_transformers

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
import pandas as pd
import numpy as np
import ast

In [25]:
# Set the maximum column width to 100 characters
pd.set_option('display.max_colwidth', 500)

In [26]:
books_df = pd.read_csv('books.csv', index_col=[0])

In [27]:
books_df.head(10)

,book_name,summaries,categories
0,The Highly Sensitive Person,"is a self-assessment guide and how-to-live template for people who feel, relate, process, and notice more deeply than others, and who frequently suffer from overstimulation as a result.",science
1,Why Has Nobody Told Me This Before?,"is a collection of a clinical psychologist’s best practical advice to combat anxiety and depression and improve our mental health in small increments, collected from over a decade of 1-on-1 work with patients.",science
2,The Midnight Library,"tells the story of Nora, a depressed woman in her 30s, who, on the day she decides to die, finds herself in a library full of lives she could have lived, where she discovers there’s a lot more to life, even her current one, than she had ever imagined.",science
3,Brave New World,"presents a futuristic society engineered perfectly around capitalism and scientific efficiency, in which everyone is happy, conform, and content — but only at first glance.",science
4,1984,is the story of a man questioning the system that keeps his futuristic but dystopian society afloat and the chaos that quickly ensues once he gives in to his natural curiosity and desire to be free.,science
5,Stolen Focus,"explains why our attention spans have been dwindling for decades, how technology accelerates this worrying trend, and what we can do to reclaim our focus and thus our capacity to live meaningful lives.",science
6,The Life-Changing Science of Detecting Bullshit,teaches its readers how to avoid falling for the lies and false information that other people spread by helping them build essential thinking skills through examples from the real world.,science
7,Dopamine Nation,"talks about the importance of living a balanced life in relation to all the pleasure and stimuli we’re surrounded with on a daily basis, such as drugs, devices, porn, gambling facilities, showing us how to avoid becoming dopamine addicts by restricting our access to them.",science
8,The Art of Statistics,"is a non-technical book that shows how statistics is helping humans everywhere get a new hold of data, interpret numbers, fact-check information, and reveal valuable insights, all while keeping the world as we know it afloat.",science
9,No Self No Problem,"is a provocative read about the implications of Buddhism in neuroscience, and more specifically about the idea that the self is only a product of the mind, meaning that there is no “I”.",science


In [28]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1227 entries, 0 to 1226
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   book_name   1227 non-null   object
 1   summaries   1227 non-null   object
 2   categories  1227 non-null   object
dtypes: object(3)
memory usage: 38.3+ KB


In [29]:
books_df.shape

(1227, 3)

## Embedding Model

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [34]:
# Step 1: Generate embeddings using BAAI/bge-small-en model from HuggingFace
model = SentenceTransformer("all-MiniLM-L6-v2")

### Exploring embedding models

In [40]:
tokenizer = model.tokenizer

In [54]:
vocab = tokenizer.get_vocab()

In [57]:
for k, v in list(vocab.items())[:10]:
    print(k, v)

whites 12461
intimidating 24439
##tt 4779
denied 6380
##ン 30263
none 3904
##pro 21572
clothes 4253
questionable 21068
italia 13052


In [61]:
sent = """non-technical book that shows how statistics"""

In [64]:
vocab['non']

2512

In [62]:
tokenizer.tokenize(sent)

['non', '-', 'technical', 'book', 'that', 'shows', 'how', 'statistics']

In [47]:
model.get_sentence_embedding_dimension()

384

In [48]:
model.encode(sent)[0:30]

array([-0.02161859, -0.01025099, -0.05919169,  0.03250577, -0.03813189,
       -0.01337735,  0.03549572,  0.05389431,  0.05898705,  0.05624209,
       -0.0607122 ,  0.05471351,  0.04883831, -0.01152746, -0.01694315,
       -0.03279695,  0.014497  ,  0.01101045, -0.01753177, -0.1471788 ,
       -0.03467362, -0.01489723,  0.07669704, -0.0206686 ,  0.06462659,
       -0.08482089, -0.02114779, -0.02981553, -0.01731552, -0.04344875],
      dtype=float32)

### Embedding the book summaries

In [66]:
# Generate embeddings for each summary
books_df['embedding'] = books_df['summaries'].map(lambda sentence: model.encode(sentence))

In [67]:
books_df.sample(5)

,book_name,summaries,categories,embedding
71,Energy,"makes you smarter by helping you understand where this important aspect of our lives comes from, how we’ve used it throughout history to get to where we are today, and why we need to be careful about how we consume it so that we can have a better future.",science,"[-0.03318367, 0.077788055, 0.08777991, 0.04545464, 0.08068968, 0.04204289, 0.120139286, -0.04293765, 0.010625668, 0.01593156, 0.038595945, 0.016533514, -0.056173064, -0.023118692, -0.022899121, -0.02144174, -0.039959576, -0.033565532, -0.093943715, -0.048956707, 0.01597185, -0.041091416, 0.07582142, 0.04578182, -0.036246803, 0.07738358, 0.01323629, -0.043080203, 0.007769628, -0.03898584, 0.072984844, 0.0089891115, 0.048759174, -0.07799531, -0.16020432, -0.01401039, 0.037216235, 0.00438509, 0..."
1208,Second Chance,"prepares you for the greatest evolutionary event in history, which we’re all living through right now: the shift from the Industrial Age to the Information Age, where jobs and traditional educations aren’t the right tools to build wealth anymore.",education,"[-0.04288759, 0.08878075, -0.010560163, 0.027145293, 0.053919684, 0.026610559, -0.021728098, -0.014199225, -0.0141423205, 0.024372116, -0.00063817686, 0.07196476, -0.036281634, -0.030289948, -0.077592, -0.02433998, -0.06277339, -0.07385383, -0.0146675855, -0.047171466, -0.04199667, 0.07095576, 0.032778177, -0.005025356, -0.024679666, 0.062312055, -0.019901084, -0.06412523, 0.027475296, -0.05628672, 0.042856563, 0.015050008, 0.083114974, 0.045182273, -0.07042723, 0.009127237, 0.07973584, 0.03..."
52,2030,"uses the current trajectory of the world, based on sociological, demographic, and technological trends, to outline the changes we can expect to happen in our lives by the beginning of the next decade.",science,"[-0.0077339825, -0.02033271, 0.02736406, 0.0041510826, 0.07891493, 0.07137656, -0.067214176, 0.020226972, -0.0009993308, -0.02200743, 0.031470392, 0.06470571, -0.0025384533, 0.07460135, -0.0036648475, -0.0330981, -0.08921485, -0.029954845, -0.01284298, 0.0049213716, -0.008718358, 0.0043923412, -0.0019282491, 0.036133543, -0.074512735, 0.007436864, 0.011357373, 0.008746888, -0.074175425, 0.038038142, 0.055553902, 0.100454815, 0.008771928, 0.025623614, -0.0828166, -0.0629245, 0.03397785, 0.038..."
1023,David and Goliath,"explains why underdogs win in situations where the odds are stacked unfavorably against them, and how you can do the same.",productivity,"[-0.05703263, -0.020235464, 0.044026, 0.063027896, 0.066035904, 0.00042772153, -0.033145525, 0.044202473, 0.113350995, 0.0012953054, -0.018177575, 0.0067520225, 0.097919516, -0.08447332, -0.023391198, -0.0013620029, 0.03144976, -0.026422331, -0.1143801, -0.029754167, 0.012016763, -0.09488206, -0.05707628, -0.040522754, 0.08093417, -0.083318405, -0.07015842, 0.048708133, -0.021562088, -0.046908382, 0.0040109577, -0.07091842, 0.06780914, -0.014243021, -0.10175364, -0.012403724, -0.074844845, 0..."
111,Feral,will help you find ways to improve the well-being of humanity by illustrating the deep connection between us and Nature and offering actionable advice on how to preserve balance in our ecosystems through rewilding.,science,"[0.035190288, 0.050729234, 0.09386551, 0.006998188, 0.007962039, 0.010932149, -0.011311008, -0.05142114, -0.027476871, -0.011129552, -0.051891875, 7.591478e-06, -0.047949247, -0.01787053, 0.03503773, 0.049290888, -0.009816475, 0.11446584, -0.07526221, 0.021548469, -0.007719987, 0.044239666, -0.038462915, 0.02072624, -0.14439924, 0.030410461, -0.04331116, 0.017407855, 0.049086694, -0.03943042, 0.02953884, 0.04896246, 0.07725295, -0.025563398, -0.07046293, 0.14759594, 0.0039173136, -0.05995152..."


## Finding topN similar books

In [68]:
# Function to find top 5 similar books based on a given title
def get_cosine_simiarity(title, df):
    book_idx = books_df[books_df['book_name'] == title].index[0]
    book_embedding = books_df.loc[book_idx, 'embedding']
    similarities = cosine_similarity([book_embedding], books_df['embedding'].tolist()).flatten()
    return  similarities

### Finding Similar Books

 - The Bitcoin Standard
 - Measure What Matters
 - The Happiness Hypothesis

In [69]:
books_df.sample(10)['book_name']

,book_name
1221,I Will Teach You To Be Rich
434,The Leader In You
502,The Go-Giver
354,Masters of Scale
513,The Fine Art Of Small Talk
533,The 5 Love Languages
922,The Design Of Everyday Things
633,Work Less Finish More
736,"Girl, Wash Your Face"
224,Titan


In [70]:
# Display the titles of the top 5 similar movies for a given movie title
#sample_name = 'Einstein: His Life And Universe'
sample_name = "The Art Of Learning"
similarity = get_cosine_simiarity(sample_name,
                                  books_df)

In [71]:
rec_books = books_df.copy()

In [72]:
rec_books['similarity'] = list(similarity)

In [73]:
rec_books.sort_values('similarity', ascending = False)[0:6]

,book_name,summaries,categories,embedding,similarity
1015,The Art Of Learning,"explains the science of becoming a top performer, based on Josh Waitzkin’s personal rise to the top of the chess and Tai Chi world, by showing you the right mindset, proper ways to practice and how to build the habits of a professional.",productivity,"[0.057849336, -0.02613431, -0.01824605, -0.04959321, -0.027506145, 0.073595524, 0.12112771, 0.07127846, 0.010800557, 0.037542548, -0.0740603, -0.029962247, 0.046538383, 0.005211696, -0.00897314, 0.01500068, 0.04359582, 0.062031116, -0.01180077, -0.12551671, -0.07901497, -0.017937625, 0.10298968, -0.033645988, -0.10056883, -0.012524447, -0.04745141, 0.018569326, 0.06537607, -0.0032507891, 0.0148648275, -0.023987692, 0.021213276, -0.0075927195, -0.06327596, 0.08779865, -0.034875814, 0.04223541...",1.000000
192,Bounce,"shows you that training trumps talent every time, by explaining the science of deliberate practice, the mindset of high performers and how you can use those tools to become a master of whichever skill you choose.",science,"[0.07816114, 0.006413545, 0.072669335, -0.040888187, -0.017057506, 0.029580545, 0.07923106, -0.00022291143, -0.022682294, -0.060730077, -0.10109995, 0.013842701, 0.018788187, -0.02829066, -0.03915965, 0.0052224677, -0.011403941, 0.017896697, -0.02605383, -0.07188093, 0.028599743, -0.03350109, 0.04188832, 0.007982287, -0.1357594, 0.043928377, -0.04126125, -0.028151141, 0.039176516, -0.036963586, 0.032730818, 0.0379503, -0.022033133, 0.02088659, -0.060343232, 0.04283302, -0.0405094, 0.0676595,...",0.627209
1064,Psyched Up,is an in-depth look at the science behind mental preparation that will show you how to do your best when it counts the most based on what top performers do.,psychology,"[0.09361925, -0.06672079, -0.004758113, -0.043322798, -0.06398673, 0.06971198, 0.01705213, 0.013314267, 0.013974421, 0.025233425, -0.08924029, 0.024607303, -0.023643296, -0.029458573, 0.017637009, 0.010701123, 0.066655606, 0.0007353872, 0.00039694208, -0.042512935, 0.012702298, -0.027440423, 0.035632662, -0.019959543, -0.041987095, 0.09196109, -0.09838528, -0.067704484, 0.049813714, 0.013659893, -0.026488965, -0.0140112, 0.05858316, -0.021636475, -0.044430416, 0.103339665, -0.043961212, 0.06...",0.593900
985,Mind Gym,"explains why the performance of world-class athletes isn’t only a result of their physical training, but just as much due to their mentally fit minds and shows you how you can cultivate the mindset of a top performer yourself.",productivity,"[0.08948291, 0.05265584, 0.018330498, -0.07779848, 0.027507726, 0.015599971, 0.07273423, 0.036996268, 0.02050743, 0.03927064, -0.054540824, 0.005562592, 0.009645464, -0.02961099, 0.033913877, -0.055170067, -0.005784571, 0.009797389, -0.07070643, 0.0013454268, -0.019187914, -0.017453654, 0.052600592, 0.04260827, -0.108015984, 0.059066877, -0.051475767, -0.012579837, -0.052748494, 0.030603442, -0.057812188, -0.036163285, -0.013445134, 0.012315827, -0.066448, 0.074971445, -0.06578766, 0.0998388...",0.581116
994,The Rise Of Superman,"decodes the science of ultimate, human performance by examining how top athletes enter and stay in a state of flow, while achieving their greatest feats, and how you can do the same.",productivity,"[0.017936006, 0.04562372, 0.007561816, -0.056011237, 0.04509487, 0.040990677, 0.011155889, -0.007352628, 0.015764415, 0.037631333, -0.12639746, -0.009437536, -0.040646758, 0.04655935, -0.03170617, -0.038416106, 0.0038976406, 0.10666279, -0.04269325, -0.0552942, 0.06701724, -0.0904257, -0.010480878, 0.055190466, -0.07316481, 0.07505966, -0.10385111, 0.011831458, -0.028691782, -0.018821148, -0.013524394, -0.03727654, 0.071204014, 0.051780283, -0.1405033, 0.06547323, -0.033143256, 0.044099633, ...",0.576935
956,Peak Performance,"shows you how to perform at your highest level by exploring the most significant factors that contribute to delivering our best work, such as stress, rest, focus, and purpose.",productivity,"[0.038

In [74]:
def get_similar_books(name, top_n = 5):
    similarity = get_cosine_simiarity(name,
                                      books_df)
    rec_books['similarity'] = list(similarity)

    return rec_books.sort_values('similarity', ascending = False)[1:top_n+1]

In [75]:
get_similar_books( 'Get A Financial Life' )

,book_name,summaries,categories,embedding,similarity
866,The Millionaire Next Door,shows you the simple spending and saving habits that lead to more cash in the bank than most people earn in their life while helping you avoid critical mistakes on your way to financial independence.,money,"[-0.009537475, 0.03984539, -0.04643179, 0.04860862, 0.044012476, -0.0036061208, 0.109966755, 0.036673076, 0.017575826, -0.0114980955, 0.012274396, -0.003930003, -0.015324689, -0.0370456, 0.030428046, -0.08801556, 0.0031616418, -0.06339551, -0.041718725, 0.009135851, -0.0026692608, -0.112013474, 0.01230945, 0.07683061, 0.004793064, 0.015984014, -0.0027925079, 0.006961843, -0.0025294637, 0.0005399654, 0.07667867, 0.06393342, 0.06954504, -0.07818694, 0.0008735532, -0.0019443599, 0.04484023, 0.1...",0.661513
832,Broke Millennial,shows those in their twenties and thirties how to manage their finances so that they can stop scraping by and instead begin to live more confidently when it comes to money.,money,"[0.01636943, 0.09722715, -0.0007254732, 0.050681647, 0.019502582, -0.0018904178, 0.0072681243, 0.0069581424, -0.037700772, 0.031602982, 0.06708081, -0.01505892, 0.025658483, -0.05237176, 0.016732052, -0.043298222, -0.048112817, -0.010302612, 0.053005744, -0.033283945, -0.0368104, -0.10629524, 0.059749465, -0.00037206567, 0.030490737, 0.062444042, -0.098911874, -0.017476004, -0.001042589, 0.07067651, 0.15693994, 0.044563603, 0.051365193, -0.00839012, -0.03387231, -0.01985746, -0.0007261552, 0...",0.651874
852,Dollars And Sense,explains why it’s so hard to manage money and teaches you how to combat false cues and natural desires so you can manage your dollars in better ways.,money,"[0.054979324, 0.043002736, -0.03762753, 0.07994845, -0.000859367, -0.0025537966, 0.087264135, 0.023318876, 0.029843777, 0.067124255, -0.008823013, 0.01745189, -0.01763041, 0.0020112174, 0.05697427, -0.07714484, -0.006483419, -0.016270662, -0.06643053, 0.031776935, 0.0450771, -0.07006652, 0.01207331, 0.0065046987, -0.0009770013, 0.06701808, -0.049844425, -0.04054092, -0.014434017, -0.03478363, 0.086386696, 0.08182275, 0.04643569, -0.043765742, -0.040282857, 0.0025635432, -0.044016756, 0.09113...",0.641765
208,More Money Than God,"teaches us about the ins and outs of hedge funds, how those managing money makes a profit, and how you can learn from them and apply their techniques to your money management strategy.",biography,"[0.05313282, -0.045197595, -0.040580623, 0.041005913, 0.013576521, -0.021785585, 0.02766448, 0.031195004, -0.052767918, 0.055406578, -0.032817546, 0.07208238, -0.0056333393, -0.016585888, 0.011596748, -0.017041752, -0.036353715, -0.0009389789, -0.028652335, -0.023240048, 0.04300677, -0.07096936, 0.037095763, -0.002504781, -0.06398876, -0.0032765674, 0.023792516, 0.012874905, -0.086575575, -0.024565216, 0.040140353, 0.0022720057, -0.025026742, -0.04367603, -0.08525991, 0.06673008, 0.001630778...",0.578497
697,The Latte Factor,teaches us how to overcome limiting beliefs about money and build our financial freedom through small daily choices.,happiness,"[0.022473317, 0.10840662, -0.06586062, 0.066976435, 0.031095685, 0.0391699, 0.057135295, -0.022181874, 0.057183806, 0.03453931, 0.024854114, 0.09547501, -0.031554423, -0.024253786, 0.016520213, -0.0597451, -0.08096308, 0.047889702, -0.01335778, -0.0069781533, -0.0045745075, -0.04366623, -0.0057079596, 0.047086634, 0.0019193545, 0.039570097, 0.07239121, -0.05961566, 0.036726985, -0.06526888, 0.06045496, 0.017766468, 0.029864794, -0.0406371, 0.02245053, 0.011211896, 0.074348986, 0.04587577, -0...",0.564691


In [76]:
books_df.sample(10, random_state = 11)

,book_name,summaries,categories,embedding
591,Happier At Home,"is an instruction manual to transform your home into a castle of happiness by figuring out what needs to be changed, what needs to stay the same, and embracing the gift of family.",relationships,"[0.0077994103, 0.09327885, 0.0045607663, 0.0058665113, -0.05553913, 0.008825402, -0.038499836, -0.04745444, -0.056372356, -0.08027113, -0.011345172, 0.11436342, 0.008592187, -0.058871027, 0.013477045, -0.048856597, -0.0711657, -0.005999426, 0.023942288, -0.0049594217, 0.022338755, 0.09703178, -0.057526667, 0.030811783, -0.023365594, 0.00060990354, 0.018511612, 0.012155563, 0.002718909, -0.053350512, 0.03228739, 0.05620759, 0.057962574, -0.035237398, -0.047036607, 0.048395097, -0.012089408, 0..."
516,Finite And Infinite Games,"offers the theory that we play many different games in life, showing you that work and relationships are long-term endeavors and how to play them in order to win.",relationships,"[-0.058282353, 0.010690715, 0.035712965, -0.047647137, 0.012895828, 0.09223728, 0.06089826, -0.020687765, 0.0601642, -0.019814376, -0.10079762, 0.02456336, -0.022688035, 0.029424084, 0.06674728, -0.016588908, -0.0030076534, -0.033017963, -0.022672033, -0.05404343, 0.0025822944, -0.08530398, -0.035254955, 0.00044275183, -0.08881852, 0.015827443, -0.024472909, 0.043738432, -0.07609073, 0.024542684, 0.004711758, 0.06318004, 0.055127643, 0.0011014704, -0.03609705, 0.056625202, -0.055506527, 0.00..."
38,Brain Food,"delves into the topic of nutrition and how certain foods and nutrients can affect the well-being of the brain, its memory function, its cognitive capability, and how what we ingest can reverse the brain’s inclination to develop certain diseases.",science,"[0.039772186, 0.027661221, 0.01944292, 0.078244954, -0.04596724, 0.119088516, 0.0828841, 0.019878546, -0.074515104, -0.010377103, -0.0596075, 0.0417529, -0.05929903, -0.07998498, -0.00037921438, -0.077793166, 0.055927128, 0.047785733, -0.10217007, 0.01945693, 0.09984964, 0.010355038, 0.023399306, 0.019309347, -0.07046333, 0.043388065, 0.052078873, -0.08543922, -0.064577095, -0.042970933, 0.113433145, -0.012125214, 0.033574797, -0.0022413048, -0.090921834, 0.0026564987, -0.035666827, -0.08094..."
194,Sex at Dawn,"challenges conventional views on sex by diving deep into our ancestors’ sexual history and the rise of monogamy, thus prompting us to rethink our understanding of what sex and relationships should really feel and be like.",science,"[-0.06714196, 0.030640874, -0.013717493, 0.026724331, -0.06720643, 0.07483334, -0.03362532, 0.007148834, 0.012260566, 0.032761406, 0.03898652, -0.0114868935, -0.06533509, -0.012363101, 0.057769313, -0.033949997, 0.009009685, -0.031863548, -0.01598496, 0.10820671, 0.0066109914, -0.034166727, 0.096837096, -0.06772517, -0.04719601, 0.012363927, -0.031339314, -0.024270969, 0.0017635805, 0.0178403, 0.00816791, 0.07720625, 0.036471907, 0.05197483, -0.08208535, 0.007448709, -0.04876371, -0.02040353..."
1142,The Long Tail,"explains why the big commercial hit is dead, how businesses can and will generate most of their future revenue from a long tail of niche products, which serve even the rarest customer needs, and what you can do to embrace this idea today.",marketing,"[-0.021558601, -0.07426368, 0.00093761523, -0.074445955, 0.06762241, 0.046283457, -0.006406186, 0.08469918, -0.019770883, -0.077788435, -0.026505377, 0.061417233, 0.015666151, -0.0015897317, 0.08591655, -0.014861997, 0.004402583, -0.024217404, -0.07089647, -0.013463666, -0.015105262, 0.018307881, -0.025101678, 0.00092173595, -0.11206358, 0.006052935, -0.017921474, -0.05324287, -0.0073297904, -0.055836182, 0.027704109, 0.097447544, 0.029030737, -0.007086584, 0.023297172, -0.00015077839, -0.02..."
566,The Longevity Project,"shows you how you can live longer by analyzing the results from one of the world’s longest-lasting studies and drawing surprising conclusions about the work ethic, happiness, love, marriage and religion of pe

In [77]:
get_similar_books( 'Permission Marketing' )

,book_name,summaries,categories,embedding,similarity
1142,The Long Tail,"explains why the big commercial hit is dead, how businesses can and will generate most of their future revenue from a long tail of niche products, which serve even the rarest customer needs, and what you can do to embrace this idea today.",marketing,"[-0.021558601, -0.07426368, 0.00093761523, -0.074445955, 0.06762241, 0.046283457, -0.006406186, 0.08469918, -0.019770883, -0.077788435, -0.026505377, 0.061417233, 0.015666151, -0.0015897317, 0.08591655, -0.014861997, 0.004402583, -0.024217404, -0.07089647, -0.013463666, -0.015105262, 0.018307881, -0.025101678, 0.00092173595, -0.11206358, 0.006052935, -0.017921474, -0.05324287, -0.0073297904, -0.055836182, 0.027704109, 0.097447544, 0.029030737, -0.007086584, 0.023297172, -0.00015077839, -0.02...",0.610961
931,Epic Content Marketing,shows why traditional methods for selling like TV and direct mail are dead and how creating content is the new future of advertising because it actually grabs people’s attention by focusing on what they care about instead of your product.,productivity,"[-0.018247752, -0.07733425, 0.01964896, -0.024510853, 0.08285824, 0.028583378, -0.050017975, 0.048232593, 0.008074594, -0.047778867, -0.0047820425, 0.110499926, -0.019245762, 0.0055096145, 0.051240258, -0.052823544, 0.039643332, -0.08599287, -0.031803828, 0.003729874, 0.04575676, 0.036747225, 0.011806236, -0.0029036985, -0.017090444, -0.03303181, -0.04751458, -0.04405505, 0.03379209, -0.041031383, 0.01096078, 0.10099078, 0.033132285, -0.02880531, -0.06683315, 0.026692336, -0.005781914, -0.04...",0.579648
1088,Purple Cow,"explains why building a great product and advertising the heck out of it simply doesn’t cut it anymore and how you can build something that’s so remarkable people have to share it, in order to succeed in today’s crowded post-advertising world.",psychology,"[-0.017602138, -0.0009882312, 0.025963686, -0.05004792, 0.12011613, 0.008536547, 0.054610707, 0.07618107, -0.01960089, -0.05813873, -0.037548397, 0.04167477, 0.020128813, -0.0082386015, 0.031226873, -0.0014111556, -0.001988202, -0.062047005, -0.071863405, -0.083956435, 0.07425073, 0.0012193805, 0.004397193, 0.048593447, -0.062215842, 0.03922052, -0.026271235, -0.01299185, 0.04055364, -0.057114843, 0.042564817, 0.019350383, 0.09867892, -0.022207702, 0.007858361, 0.033683673, -0.044208612, -0....",0.577623
1126,This Is Marketing,"argues that marketing success in today’s world comes from focusing more on the needs, values, and desires of our target audience, rather than spamming as many people as possible with our message.",marketing,"[0.021531057, -0.0069928495, -0.007194959, -0.021298204, 0.0699725, 0.0033442064, 0.1252375, 0.0131815765, 0.025552971, -0.14000848, 0.039742075, 0.013115402, 0.07759705, -0.0299957, 0.13576858, -0.03337302, 0.05219282, -0.018406201, -0.07159453, 0.010958593, -0.06683714, -0.0070391805, -0.000850616, 0.030236293, -0.12841378, -0.077462554, -0.042098537, -0.02542872, -0.00837806, -0.031613797, 0.037909918, 0.038204167, 0.06642599, 0.01868012, 0.053969093, 0.048373595, -0.07059402, -0.02528964...",0.575825
1132,The Fortune Cookie Principle,"explains why a great product or service isn’t enough, how you can tell a compelling story about your brand and why that’s the most important aspect of running a business today.",marketing,"[0.02274764, 0.029120019, 0.044820976, -0.07860731, 0.05329564, 0.025894986, 0.039052192, 0.09677936, -0.0011926354, -0.05489164, -0.024087168, 0.094900906, 0.011908747, -0.0020536669, 0.052389696, -0.06703157, 0.070614964, -0.056515466, -0.09647857, -0.051603265, 0.08005688, -0.03946483, 0.02467629, 0.08539038, -0.11195082, 0.04220527, 0.014462373, 0.009545159, -0.03881597, -0.10019578, -0.043126445, 0.038978238, 0.06593385, -0.05899836, 0.028070511, 0.0014222747, 0.0010388768, -0.002432501...",0.552454
